![sql](img/sql-logo.jpg)

# A beginner's guide to databases, SQL, & using them with `pandas`

**Scenario:** You are a data analyst for the Homeland Security, trying to create reports on the active airports world wide. The data you need to access is in a SQL database. YOu need to be able to query for the data in a database!

## Learning goals:
- Goal 1: Summarize the use case for sql in the data science skill set
- Goal 2: Define key sql terminology
- Goal 3: Get information about DB schema and table structure
- Goal 4: Use basic SQL commands:
    - Construct SQL queries
    - Use JOIN to merge tables along logical columns
    - Grouping Data with SQL
- Goal 5: Query data from pandas dataframes using SQL
- Goal 6: Convert SQL to pandas

## Goal 1: Summarize

[Netflix has a great article](https://medium.com/netflix-techblog/notebook-innovation-591ee3221233) describing three different data roles at their company, their different needs, and their toolsets.

![netflix](img/netflix-data-roles.jpeg)

Examining that graphic, SQL shows up as one of the tools of the _Data Engineer_ 

Data Engineers provide the essential data architecture services that make data science possible.

![hierarchy](img/ai-hierachy.png)

[Source: Monica Rogati’s fantastic Medium post “The AI Hierarchy of Needs”
](https://hackernoon.com/the-ai-hierarchy-of-needs-18f111fcc007)

![etl](img/etl.png)

[img source: Jeff Hammerbacher’s slide from UC Berkeley CS 194 course ](https://bcourses.berkeley.edu/courses/1377158/pages/cs-194-16-introduction-to-data-science-fall-2015)

### What is a Relational Database? 

![rdb](img/relational-dbms-model.png)
[reference for image ](https://www.studytonight.com/dbms/database-model.php)

### POPULAR RDBMS

- SQLite
- MySQL
- PostgreSql
- Oracle DB
- SQL Server

***
## Goal 2: Database terminology

### Relational Database Schema

![schema](img/MySQL_Schema_Music_Example.png)

[source of image](https://database.guide/what-is-a-database-schema/)

### Table columns view
![table example](img/columns.png)

### Terminology

- Schema
- Primary Key
- Foreign Key
- Structured queries
- Views

***
### SQLite

![sqlite](img/SQLite-Python.jpg)

"SQLite is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language. Some applications can use SQLite for internal data storage. It’s also possible to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle." - [sqlite documentation](https://docs.python.org/2/library/sqlite3.html)



## But what about connecting to database servers?

To connect to an Oracle database server there is a different package, `cx_Oracle`. The documentation for that package is [here](https://oracle.github.io/python-cx_Oracle/)

***
## Goal 3: Get going with sqlite!

In [1]:
import sqlite3

#### Load a DB object with `connect` and `cursor`

In [5]:
con = sqlite3.connect('flights.db')
cursor = con.cursor()

#### Use sqlite_master to find all the tables in the schema
Get the schema of a database from a db in sqlite

In [20]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('airports',), ('airlines',), ('routes',)]


In [21]:
cursor.execute("SELECT name, type FROM sqlite_master;")
print(cursor.fetchall())

[('airports', 'table'), ('ix_airports_index', 'index'), ('airlines', 'table'), ('ix_airlines_index', 'index'), ('routes', 'table'), ('ix_routes_index', 'index')]


In [23]:
cursor.execute("SELECT * FROM airports")

In [24]:
cursor.execute("SELECT * FROM sqlite_master;")
results = cursor.fetchall()
for res in results:
    print(res)

('table', 'airports', 'airports', 2, 'CREATE TABLE airports (\n[index] INTEGER,\n  [id] TEXT,\n  [name] TEXT,\n  [city] TEXT,\n  [country] TEXT,\n  [code] TEXT,\n  [icao] TEXT,\n  [latitude] TEXT,\n  [longitude] TEXT,\n  [altitude] TEXT,\n  [offset] TEXT,\n  [dst] TEXT,\n  [timezone] TEXT\n)')
('index', 'ix_airports_index', 'airports', 3, 'CREATE INDEX ix_airports_index ON airports ([index])')
('table', 'airlines', 'airlines', 945, 'CREATE TABLE airlines (\n[index] INTEGER,\n  [id] TEXT,\n  [name] TEXT,\n  [alias] TEXT,\n  [iata] TEXT,\n  [icao] TEXT,\n  [callsign] TEXT,\n  [country] TEXT,\n  [active] TEXT\n)')
('index', 'ix_airlines_index', 'airlines', 946, 'CREATE INDEX ix_airlines_index ON airlines ([index])')
('table', 'routes', 'routes', 1393, 'CREATE TABLE routes (\n[index] INTEGER,\n  [airline] TEXT,\n  [airline_id] TEXT,\n  [source] TEXT,\n  [source_id] TEXT,\n  [dest] TEXT,\n  [dest_id] TEXT,\n  [codeshare] TEXT,\n  [stops] TEXT,\n  [equipment] TEXT\n)')
('index', 'ix_routes_i

#### Get information about one table

**A note about** `execute`<br>
Each time you use it, you reset the value of cursor

In [25]:
cursor.execute("SELECT * FROM airports as a").fetchone()

(0,
 '1',
 'Goroka',
 'Goroka',
 'Papua New Guinea',
 'GKA',
 'AYGA',
 '-6.081689',
 '145.391881',
 '5282',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [26]:
cursor.fetchone()

(1,
 '2',
 'Madang',
 'Madang',
 'Papua New Guinea',
 'MAG',
 'AYMD',
 '-5.207083',
 '145.7887',
 '20',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [27]:
cursor.fetchmany(4)

[(2,
  '3',
  'Mount Hagen',
  'Mount Hagen',
  'Papua New Guinea',
  'HGU',
  'AYMH',
  '-5.826789',
  '144.295861',
  '5388',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (3,
  '4',
  'Nadzab',
  'Nadzab',
  'Papua New Guinea',
  'LAE',
  'AYNZ',
  '-6.569828',
  '146.726242',
  '239',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (4,
  '5',
  'Port Moresby Jacksons Intl',
  'Port Moresby',
  'Papua New Guinea',
  'POM',
  'AYPY',
  '-9.443383',
  '147.22005',
  '146',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (5,
  '6',
  'Wewak Intl',
  'Wewak',
  'Papua New Guinea',
  'WWK',
  'AYWK',
  '-3.583828',
  '143.669186',
  '19',
  '10',
  'U',
  'Pacific/Port_Moresby')]

#### Use description

In [30]:
cursor.description

(('index', None, None, None, None, None, None),
 ('id', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('city', None, None, None, None, None, None),
 ('country', None, None, None, None, None, None),
 ('code', None, None, None, None, None, None),
 ('icao', None, None, None, None, None, None),
 ('latitude', None, None, None, None, None, None),
 ('longitude', None, None, None, None, None, None),
 ('altitude', None, None, None, None, None, None),
 ('offset', None, None, None, None, None, None),
 ('dst', None, None, None, None, None, None),
 ('timezone', None, None, None, None, None, None))

#### Or use `Pragma`
`Pragma` tool [link here](https://www.sqlite.org/pragma.html#pragma_table_info)

**output**<br>
`(column id, column name, data type, whether or not the column can be NULL, and the default value for the column)`

In [14]:
cursor.execute("PRAGMA table_info(airports)")
info = cursor.fetchall()
print(*info, sep = "\n") 

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'city', 'TEXT', 0, None, 0)
(4, 'country', 'TEXT', 0, None, 0)
(5, 'code', 'TEXT', 0, None, 0)
(6, 'icao', 'TEXT', 0, None, 0)
(7, 'latitude', 'TEXT', 0, None, 0)
(8, 'longitude', 'TEXT', 0, None, 0)
(9, 'altitude', 'TEXT', 0, None, 0)
(10, 'offset', 'TEXT', 0, None, 0)
(11, 'dst', 'TEXT', 0, None, 0)
(12, 'timezone', 'TEXT', 0, None, 0)


#### Making fetch happen

`.fetchall()` is how you get the query results out of the object.

You can also `.fetchone()` or `.fetchmany()`

**Task:** Get the descriptive data for airlines and routes tables

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'alias', 'TEXT', 0, None, 0)
(4, 'iata', 'TEXT', 0, None, 0)
(5, 'icao', 'TEXT', 0, None, 0)
(6, 'callsign', 'TEXT', 0, None, 0)
(7, 'country', 'TEXT', 0, None, 0)
(8, 'active', 'TEXT', 0, None, 0)


***
## Goal 4: Use basic SQL commands 
- Construct SQL queries
- Use JOIN to merge tables along logical columns
- Grouping Data with SQL

In [32]:
cursor.execute("""select "Flatiron School\"""").fetchall()

[('Flatiron School',)]

### Construct SQL queries

**Select**

**From**

**Where**

**Order by**

**Limit**

#### Options for each:

**Select**:  `distinct`, using `as` to rename columns, single number aggregates

**From:** also uses aliasing with `as`

**Where**: equals is only one `=`, `between`, `in`, wildcards `%`, `AND`, `OR`, `NOT`

**Order by**: `asc` and `desc`

**Limit**:  #

**Task**: 
- Select only active airlines in the UK from the airlines table
- Select the unique list of countries with airports

In [33]:
cursor.execute("""SELECT distinct country 
                  FROM airlines""").fetchall()

[(None,),
 ('United States',),
 ('South Africa',),
 ('United Kingdom',),
 ('Russia',),
 ('Thailand',),
 ('Canada',),
 ('Australia',),
 ('Singapore',),
 ('Belgium',),
 ('Mexico',),
 ('Spain',),
 ('France',),
 ('United Arab Emirates',),
 ('Republic of Korea',),
 ('Pakistan',),
 ('Libya',),
 ('Gambia',),
 ('Ivory Coast',),
 ('Ukraine',),
 ('Democratic Republic of the Congo',),
 ('Iran',),
 ('Finland',),
 ('Brazil',),
 ('Colombia',),
 ('AEROCENTER',),
 ('Ghana',),
 ('Kenya',),
 ('Liberia',),
 ('Togo',),
 ('Somali Republic',),
 ('Morocco',),
 ('Canadian Territories',),
 ('Dominican Republic',),
 ('Japan',),
 ('Albania',),
 ('Nigeria',),
 ('Germany',),
 ('Slovenia',),
 ('Czech Republic',),
 ('Benin',),
 ('AEROCESAR',),
 ('Greece',),
 ('Chile',),
 ('Tanzania',),
 ('Bolivia',),
 ('Italy',),
 ('Sweden',),
 ('Argentina',),
 ('Sierra Leone',),
 ('Indonesia',),
 ('Senegal',),
 ('Afghanistan',),
 ('Uganda',),
 ('Bosnia and Herzegovina',),
 ('Gabon',),
 ('Angola',),
 ('Uzbekistan',),
 ('Namibia',),


In [35]:
cursor.execute("""
;
""")
cursor.fetchmany(5)

[]

### SQL Joins

SQL joins can be used to both **add** data to a table and **remove** data from a table. 

![venn](img/venn.png)

**Task** Write a query that will join the latitude and longitude data from the airports table to the information on the routes table

In [37]:
cursor.execute("""
SELECT *
FROM routes
""")
print(*cursor.description, sep='\n')

('index', None, None, None, None, None, None)
('airline', None, None, None, None, None, None)
('airline_id', None, None, None, None, None, None)
('source', None, None, None, None, None, None)
('source_id', None, None, None, None, None, None)
('dest', None, None, None, None, None, None)
('dest_id', None, None, None, None, None, None)
('codeshare', None, None, None, None, None, None)
('stops', None, None, None, None, None, None)
('equipment', None, None, None, None, None, None)


In [36]:
cursor.execute("""
SELECT *
FROM airports
""")
print(*cursor.description, sep='\n')

('index', None, None, None, None, None, None)
('id', None, None, None, None, None, None)
('name', None, None, None, None, None, None)
('city', None, None, None, None, None, None)
('country', None, None, None, None, None, None)
('code', None, None, None, None, None, None)
('icao', None, None, None, None, None, None)
('latitude', None, None, None, None, None, None)
('longitude', None, None, None, None, None, None)
('altitude', None, None, None, None, None, None)
('offset', None, None, None, None, None, None)
('dst', None, None, None, None, None, None)
('timezone', None, None, None, None, None, None)


In [38]:
cursor.execute(""" """).fetchone()


('-6.081689', '145.391881', 'CG', 'GKA', 'HGU')

### Grouping statements

Combines `select` and `group by` when you want aggregates by values

`select` `min(x)` ... `max()`, `sum()`, etc

`group by x`

**Task**<br>
- Which countries have the highest amount of active airlines?
- Which countries have the highest amount of inactive airlines?
- What about airports by timezones?

In [39]:
# Which countries have the highest amount of active airlines?
cursor.execute("""
SELECT country, count(*) AS count
FROM airlines
WHERE active = 'Y'
GROUP BY country
ORDER BY count DESC
LIMIT 10
""")
print(*cursor.fetchall(), sep='\n')

('United States', 141)
('Russia', 72)
('United Kingdom', 40)
('Germany', 37)
('Canada', 34)
('Australia', 26)
('China', 25)
('Spain', 24)
('Brazil', 23)
('France', 22)


In [40]:
# Which countries have the highest amount of inactive airports?
cursor.execute("""
SELECT country, count(*) AS count
FROM airlines
WHERE active = 'N'
GROUP BY country
ORDER BY count DESC
LIMIT 10
""")
print(*cursor.fetchall(), sep='\n')

('United States', 939)
('Mexico', 427)
('United Kingdom', 367)
('Canada', 284)
('Russia', 158)
('Spain', 142)
('France', 97)
('Germany', 94)
('South Africa', 81)
('Nigeria', 80)


In [ ]:
# What about airports by timezones?
cursor.execute("""

""")
print(*cursor.fetchall(), sep='\n')

In [ ]:
# It's always a good idea to close our connections when we're done
cursor.close()
con.close()

## Goal 5: Using sql within pandas to filter

`.query()`

[query documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html)

In [ ]:
# Get data for an example
import pandas as pd
shelter_data=pd.read_csv('https://data.austintexas.gov/api/views/9t4d-g238/rows.csv?accessType=DOWNLOAD')
shelter_data.dtypes

In [ ]:
max_data = shelter_data.query('Name == "Max"')
max_data.head()
#vs
shelter_data[shelter_data['Name']=="Max"]

In [ ]:
shelter_data.rename(index=str, columns={"Animal Type": "animal_type"}, inplace = True)
test = shelter_data.query('animal_type == "Dog"')
test.head()

## Goal 6: Transfering from sqlite to pandas

In [ ]:
pd_con = sqlite3.connect("flights.db")
df = pd.read_sql_query("select * from airports limit 5;", pd_con)
df

**Task**: 
Convert one of the earlier queries in the lesson to a pandas data frame

In [ ]:
# Which countries have the highest amount of active airports?
df = pd.read_sql_query("""
SELECT country, count(*) AS count
FROM airlines
WHERE active = 'Y'
GROUP BY country
ORDER BY 2 DESC
LIMIT 10
""", pd_con)
df

## Integration

The [Chinook database](https://github.com/lerocha/chinook-database) is a sample database, representing a digital media store.

You need to create a query that can rank tracks in term of popularity.

The name of the database is `Chinook_Sqlite.sqlite`

Database information:<br>
- How many tables are in the database?
- What's the primary key of each table?
- What foreign keys join the tables together?
- If you had to draw a schema of how the tables are connected, what would it look like?

To answer the question:<br>
- What are the max and min dates in the Invoice table?
- What tables would you need to answer "what is your most popular track?"
- What values from each table?

In [ ]:
conn = sqlite3.connect('Chinook_Sqlite.sqlite')
cur = conn.cursor()

In [ ]:
# How many tables are in the database?
cur.execute("SELECT count(*) as table_count FROM sqlite_master WHERE type = 'table';").fetchone()

In [ ]:
# What's the primary key of each table?
table_names = cur.execute("SELECT name FROM sqlite_master WHERE type = 'table';").fetchall()
for table_name in table_names:
    tname = table_name[0]
    # table_name is still a tuple of length 1, so we have to unpack to get the table name value
    results = cur.execute(f"PRAGMA table_info({tname});").fetchall()

    # column name is 2nd column,
    # primary key is the 4th column, according to https://www.sqlite.org/pragma.html#pragma_table_info
    pk_column_name = [x[1] for x in results if x[3] == 1]
    print(f"{tname}.{pk_column_name[0]}")

In [ ]:
# What foreign keys join the tables together?
table_names = cur.execute("SELECT name FROM sqlite_master WHERE type = 'table';").fetchall()
for table_name in table_names:
    tname = table_name[0]
    # table_name is still a tuple of length 1, so we have to unpack to get the table name value
    results = cur.execute(f"PRAGMA foreign_key_list({tname});").fetchall()
    if results:
        print(tname, 'table:')
        print(*results, sep='\n')

In [ ]:
# What are the max and min dates in the Invoice table?
cur.execute("""
SELECT max(InvoiceDate) AS max_date,
min(InvoiceDate) AS min_date
FROM Invoice""").fetchall()

In [ ]:
# What tables would you need to answer "what is your most popular track?"

# Track (for track info)
# InvoiceLine (for track puchase quantity)

In [ ]:
# What values from each table?

# Track.TrackId, Track.Name, Track.AlbumId
# InvoiceLine.TrackId, InvoiceLine.Quantity
# Album.AlbumId, Album.ArtistId
# Artist.ArtistId, Artist.Name

In [ ]:
# Put it all together:
# You need to create a query that can rank tracks in term of popularity.

results = cur.execute("""
SELECT
t.name as track_name,
sum(il.quantity) AS purchase_count
FROM Track t
JOIN InvoiceLine il USING (TrackId)
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10
""").fetchall()

print(*results, sep='\n')

In [ ]:
# Advanced: get the artist who sang the song!
results = cur.execute("""
SELECT
t.name as track_name,
a.name as artist_name,
sum(il.quantity) AS purchase_count
FROM Track t
JOIN InvoiceLine il USING (TrackId)
JOIN Album USING (AlbumId)
JOIN Artist a USING (ArtistId)
GROUP BY 1, 2
ORDER BY 3 DESC
LIMIT 10
""").fetchall()

print(*results, sep='\n')

## Reflection